<a href="https://colab.research.google.com/github/JosegSuarezb/Integracion-de-los-datos/blob/main/Reto_4_Integracion_Dinamica_Jose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

De acuerdo con la base de datos (4. DB_Fintechs (USD)), llevar a cabo la integración dinámica incremental de los datos para las Fintechs que hacen parte de la base de datos. Para cada integración a la base de datos original, mostrar los valores de los cluster iniciales (K-Means), los valores de los clusters luego de la integración de los datos (K-Medoids), la variabilidad de los valores de clusters, así como el número de datos que se integraron por cluster para la base de datos interna. Para la construcción de los clusters, es importante tener en cuenta las variables Edad, Ingresos, Egresos, y Monto; así como un total de cinco (5) clusters por agrupación. Aquí, es importante destacar que la integración se tenga que hacer como minimo tres veces, por lo que es importante el desarrollo de los métodos para tal fin.

##Carga librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def integracion_dinamica(XCd, XDBe):
  """
  Integra datos dinámicamente usando K-Medoids.

  Args:
    XCd: DataFrame con los centroides de los clusters iniciales (K-Means).
    XDBe: DataFrame con los datos externos a integrar.

  Returns:
    Un diccionario con la información de la integración.
  """

  XCe = XCd.copy()  # Copia de los centroides iniciales

  NDe = np.zeros((len(XCd), 1))  # Inicializar contador de datos por cluster
  nf = np.zeros((len(XDBe), 1))  # Inicializar asignación de clusters

  for k in range(len(XDBe)):
    # Changed line: Removed .values since XCe and XDBe[k] are NumPy arrays
    de = np.sqrt(np.sum((XCe - XDBe[k])**2, axis=1))  # Distancia a centroides
    fila = np.argmin(de)  # Cluster más cercano
    nf[k,] = fila
    XCe[fila,] = (XCe[fila,] + XDBe[k,:]) / 2  # Actualizar centroide
    NDe[fila,] += 1  # Incrementar contador

  # Calcular la variabilidad de los clusters
  variabilidad=(XCd-XCe)/XCe

  # Crear un diccionario con la información de la integración
  resultados = {
      'centroides_iniciales': XCd,
      'centroides_finales': XCe,
      'variabilidad': variabilidad,
      'datos_por_cluster': NDe.flatten().astype(int),
      'asignacion_clusters': nf.flatten().astype(int)
  }
  return resultados

## Carga de la base de datos

In [ ]:
nxl='/content/drive/MyDrive/Integracion de datos (enfasis)/4. DB_Fintechs (USD).xlsx'
XDB=pd.read_excel(nxl,sheet_name=0) #base de datos observada
XDBe1=pd.read_excel(nxl,sheet_name=1) #base de datos externa 1
XDBe2=pd.read_excel(nxl,sheet_name=2) #base de datos externa 2
XDBe3=pd.read_excel(nxl,sheet_name=3) #base de datos externa 3


XDB=XDB.iloc[:, [1, 10, 11, 25]].dropna()
XDB=np.array(XDB)
XDBe1=XDBe1.iloc[:, [1, 10, 11, 25]].dropna()
XDBe1=np.array(XDBe1)
XDBe2=XDBe2.iloc[:, [1, 10, 11, 25]].dropna()
XDBe2=np.array(XDBe2)
XDBe3=XDBe3.iloc[:, [1, 10, 11, 25]].dropna()
XDBe3=np.array(XDBe3)


##Proceso de clusterizacion estatica

In [ ]:
#1. se deben crear las semillas iniciales
XDBM=np.max(XDB,axis=0) #axis=0 que lo haga por columna
XDBm=np.min(XDB,axis=0)

XC=np.zeros((5,4))

for i in range(5): #numero de montoncitos
  for j in range(4): #cada montoncito va a tener 4 variables
    XC[i,j]=np.random.uniform(XDBm[j],XDBM[j])

print(XC.astype(int))

XDB=np.array(XDB)
XCd=np.zeros((5,4)) #los clusters despues
ND=np.zeros((5,1))

#2. se procede con la clasificacion de los datos por motoncitos
for k in range(len(XDB)):
  d=np.sum(np.abs(XC[:,:]-XDB[k,:]),axis=1) #distancia de un registro a un cluster
  fila=np.argmin(d) #numero de cluster al que pertence un dato

  XCd[fila,]=XCd[fila,]+XDB[k,:] #voy sumando la informacion de las variables a los clusters
  ND[fila,]=ND[fila,]+1 #cuento el numero de datos que quedaron por cluster
  ND[ND == 0] += 1
print('El numero de datos por cluster es:',ND)

XCd=XCd/ND #dividimos todas las variables por el numero de datos del cluster

dfxco=pd.DataFrame((XCd))
dfxco.columns=['Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']
dfxco.head(5)

# se procede a mostrar la correlacion entre las variables
dfcor=pd.DataFrame((XDB)).corr()
print('La correlacion entre las variables es: \n',dfcor)

[[  42 2089 1200 2214]
 [  31 1689 1441 2012]
 [  23  734 2520 1774]
 [  68 2355 2914  910]
 [  48 1662 2364 1832]]
El numero de datos por cluster es: [[ 21.]
 [536.]
 [894.]
 [ 27.]
 [ 26.]]
La correlacion entre las variables es: 
           0         1         2         3
0  1.000000 -0.033098 -0.037194 -0.039985
1 -0.033098  1.000000  0.888146  0.818815
2 -0.037194  0.888146  1.000000  0.522718
3 -0.039985  0.818815  0.522718  1.000000


##Integracion dinamica finca 1

In [ ]:
# Realizar la integración dinámica tres veces
for i in range(3):
  resultados = integracion_dinamica(XCd, XDBe1)
  XCe1 = resultados['centroides_finales']  # Actualizar centroides para la siguiente iteración

  print(f"Integración {i + 1}:")
  print("Centroides iniciales:", resultados['centroides_iniciales'])
  print("Centroides finales:", resultados['centroides_finales'])
  print("Variabilidad:", resultados['variabilidad'])
  print("Datos por cluster:", resultados['datos_por_cluster'])
  print("-" * 20)

#el informe de integracion es el siguiente:
dfinf=pd.DataFrame(np.column_stack((resultados["asignacion_clusters"],XDBe1)))
dfinf.columns=['Cluster','Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']
dfinf.head(10)

Integración 1:
Centroides iniciales: [[  36.9047619  1874.6372619  1018.71839153 2140.42330219]
 [  41.05783582 1141.52043797  971.8744631   854.39397258]
 [  41.19574944  403.38249413  355.0975123   298.97512196]
 [  38.11111111 2043.33674074 2344.90967901  953.59930715]
 [  38.34615385 1718.35768269 1916.61844444  987.97169677]]
Centroides finales: [[  39.40490003 2045.73092694 2321.89221115 1065.20583822]
 [  42.7471083   231.62733443  235.20957685  134.70785452]
 [  46.40785322 1212.23267514  721.62720604 1578.25752697]
 [  48.81246169 1817.97917014  928.69998441 2669.07199287]
 [  31.71873225 1079.250088    907.84395254  726.50371765]]
Variabilidad: [[-0.06344739 -0.08363449 -0.56125509  1.00939877]
 [-0.03951782  3.92826307  3.13195107  5.34256982]
 [-0.11231081 -0.66724004 -0.50792111 -0.81056633]
 [-0.21923399  0.12396048  1.52493778 -0.64272252]
 [ 0.20894346  0.59217748  1.11117609  0.35989902]]
Datos por cluster: [ 63 527 533 142 236]
--------------------
Integración 2:
Cent

,Cluster,Edad,Ingresos,Egresos,Monto (EAD)
0,2.0,73.0,153.20550,195.844222,84.871944
1,2.0,31.0,105.00425,93.578222,106.112376
2,2.0,37.0,185.24475,167.963778,137.737842
3,2.0,57.0,609.92800,559.075556,315.196041
4,2.0,31.0,344.88900,290.041556,256.987902
5,4.0,64.0,1248.61700,1589.392667,755.113136
6,2.0,41.0,157.64225,125.214889,146.714016
7,4.0,37.0,1490.97350,1813.193556,559.446490
8,1.0,66.0,984.91125,674.762000,704.453317
9,1.0,46.0,1239.47525,571.404222,1623.010404


##Integracion finca 2

In [ ]:
# Realizar la integración dinámica tres veces
for i in range(3):
  resultados = integracion_dinamica(XCd, XDBe2)
  XCe2 = resultados['centroides_finales']  # Actualizar centroides para la siguiente iteración

  print(f"Integración {i + 1}:")
  print("Centroides iniciales:", resultados['centroides_iniciales'])
  print("Centroides finales:", resultados['centroides_finales'])
  print("Variabilidad:", resultados['variabilidad'])
  print("Datos por cluster:", resultados['datos_por_cluster'])
  print("-" * 20)

#el informe de integracion es el siguiente:
dfinf=pd.DataFrame(np.column_stack((resultados["asignacion_clusters"],XDBe2)))
dfinf.columns=['Cluster','Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']
dfinf.head(10)

Integración 1:
Centroides iniciales: [[  36.9047619  1874.6372619  1018.71839153 2140.42330219]
 [  41.05783582 1141.52043797  971.8744631   854.39397258]
 [  41.19574944  403.38249413  355.0975123   298.97512196]
 [  38.11111111 2043.33674074 2344.90967901  953.59930715]
 [  38.34615385 1718.35768269 1916.61844444  987.97169677]]
Centroides finales: [[  28.49788128 1357.98725535 1261.83226453  861.02010913]
 [  43.53483473 1604.10323713  868.24577827 2353.78650902]
 [  33.07465273  740.35535815  806.33389818  465.30435913]
 [  36.15582887 1891.14504501 1992.73177251 1085.65025176]
 [  43.06996254  201.15182843  155.12209512  195.29245166]]
Variabilidad: [[ 0.2950002   0.38045277 -0.19266735  1.48591558]
 [-0.05689694 -0.28837471  0.11935409 -0.63701297]
 [ 0.24553838 -0.45515017 -0.5596148  -0.35746331]
 [ 0.05407931  0.08047595  0.17673121 -0.12163304]
 [-0.10967757  7.54259042 11.35554769  4.05893437]]
Datos por cluster: [126 197 539  98 540]
--------------------
Integración 2:
Cent

,Cluster,Edad,Ingresos,Egresos,Monto (EAD)
0,1.0,24.0,929.57975,996.479556,545.193466
1,2.0,53.0,178.29025,145.511111,116.989171
2,2.0,37.0,613.52050,415.619556,673.152608
3,1.0,50.0,1167.62800,1070.444889,1106.561066
4,1.0,32.0,1350.33750,1281.001778,544.182398
5,2.0,22.0,361.60800,383.308444,218.684938
6,2.0,22.0,9.10200,9.511111,5.276556
7,1.0,33.0,889.93200,1014.307333,522.734021
8,2.0,26.0,172.04825,150.991111,91.443033
9,2.0,43.0,266.39275,322.597556,167.074889


##Integracion finca 3

In [ ]:
# Realizar la integración dinámica tres veces
for i in range(3):
  resultados = integracion_dinamica(XCd, XDBe3)
  XCe3 = resultados['centroides_finales']  # Actualizar centroides para la siguiente iteración

  print(f"Integración {i + 1}:")
  print("Centroides iniciales:", resultados['centroides_iniciales'])
  print("Centroides finales:", resultados['centroides_finales'])
  print("Variabilidad:", resultados['variabilidad'])
  print("Datos por cluster:", resultados['datos_por_cluster'])
  print("-" * 20)

#el informe de integracion es el siguiente:
dfinf=pd.DataFrame(np.column_stack((resultados["asignacion_clusters"],XDBe3)))
dfinf.columns=['Cluster','Edad', 'Ingresos', 'Egresos', 'Monto (EAD)']
dfinf.head(10)

Integración 1:
Centroides iniciales: [[  36.9047619  1874.6372619  1018.71839153 2140.42330219]
 [  41.05783582 1141.52043797  971.8744631   854.39397258]
 [  41.19574944  403.38249413  355.0975123   298.97512196]
 [  38.11111111 2043.33674074 2344.90967901  953.59930715]
 [  38.34615385 1718.35768269 1916.61844444  987.97169677]]
Centroides finales: [[  38.25283719 1099.18468288  568.00087327 1307.05233579]
 [  48.71578081  892.65604917  624.3765533   590.9657427 ]
 [  40.75975337  255.94466799  194.57152274  240.3188589 ]
 [  54.54303794 1915.56795379 1610.84141326 1696.55361218]
 [  30.86997785 1429.35770399 1284.90154231  778.41770954]]
Variabilidad: [[-0.03524118  0.70547979  0.79351554  0.63759571]
 [-0.15719639  0.27879091  0.55655182  0.44575888]
 [ 0.01069673  0.57605352  0.82502304  0.24407682]
 [-0.30126534  0.06670021  0.45570486 -0.43791973]
 [ 0.24218275  0.2021887   0.49164615  0.26920506]]
Datos por cluster: [291 297 579  46 129]
--------------------
Integración 2:
Cent

,Cluster,Edad,Ingresos,Egresos,Monto (EAD)
0,2.0,47.0,562.66000,380.999333,490.409628
1,2.0,42.0,806.36650,359.774444,974.824904
2,2.0,26.0,711.37025,700.833111,462.887434
3,3.0,50.0,2271.35575,2601.199111,1177.188187
4,2.0,26.0,361.71000,470.925111,244.151411
5,0.0,61.0,1461.36500,846.371111,1673.809324
6,2.0,66.0,637.96325,604.930889,396.736971
7,2.0,34.0,132.52500,104.086444,138.267139
8,1.0,42.0,797.48950,893.312000,369.054041
9,1.0,48.0,785.91100,789.580000,450.349741


#**Analisis de resultados**

De acuerdo con la base de datos (DB_Fintechs (USD)), llevar a cabo la integración dinámica incremental de los datos para las Fintechs que hacen parte de la base de datos. Se tomaron como variables a estudiar la Edad, los Ingresos, los Egresos y el Monto (EAD). Se realizo primero el proceso de clusterizacion estatica con la base de datos clave. Se identifico que el cluster al que mas se le distribuyeron datos fue al cluster 3 con un total de 894 datos. Además, se identifico que las variables que tienen mayor correlacion entre son Ingresos y Egresos con 0.88 lo que tiene mucha incidencia que si hay mas ingresos hay mayores gastos.

Ahora bien luego se realizo la integracion de cada finca exterior con la finca base. Para la integracion de la Finca 1 se encontro que el cluster que mas almaceno datos fue el cluster 3 con 533 datos. Además, se encontro que el cluster dos fue el que mas variabilidad tuvo ya que la diferencia de los centroides iniciales con los finales fueron: en la variable ingresos disminuyo un 392%, en egresos un 313% y en Monto un 534% lo que no seria considerable realizar la integracion por la diferencia tan grande.

Para la integracion de la finca dos se identifico que el cluster que mas almaceno datos fue el cluster 5 con un total de 540 datos. Sin embargo, se observo que este cluster tambien tuvo una gran variabilidad en comparacion con los centroides iniciales con los finales en las variables de Ingreso con una disminucion del 754%, Egresos con una disminucion del 1135% y Monto con una disminucion del 405%. Lo que teniendo en cuenta que este es el cluster que mas almaceno datos no seria bueno integrar porque habra mucha diferencia con el inicial.

Por ultimo para la integracion de la finca tres se identifico que el cluster 3 fue el que mas importancia tuvo con un almacenamiento total de 579 datos. Observando los datos arrojados se puede considerar esta integracion como buena en comparación con las demás debido a que en esta no hubo diferencias mayores al 100%, en la unica que se observo variaciones significativas fue en el cluster 1 donde hubo diferencias entre los centroides iniciales con los finales en las siguientes variables: Ingresos con una disminución del 70%, Egresos con una disminución del 79% y Monto con una disminución del 63%.
